# Approximate H2 molecule energy near ground state

In this example notebook, we'll walk through calculating the energy of a molecule given an estimate for the ground state using Azure Quantum. We will do that using a simple algorithm where we prepare the quantum register in a state near the ground state of the molecule using the Jordan-Wigner encoding, and evaluate the energy of the molecule by applying the Hamiltonian operator to that state. To be able to run the algorithm on near-term hardware, we will estimate the energy per Hamiltonian term, and then calculate the total estimated energy by adding the values of these terms.

In [51]:
# First, import qsharp and qsharp.azure to be able to compile and submit the quantum program.
import qsharp
import qsharp.azure
qsharp.reload()
from Microsoft.Quantum.Chemistry.Hamiltonian import GetHamiltonianTerm, GetHamiltonianTermH2, ExpandedCoefficients_, JordanWignerMeasurementOperators


Reloading workspace.

### 1. Load H2 molecule data

The pre-generated Broombridge file included in this sample contains the details on the molecule's Hamiltonian. A Hamiltonian is an operator that calculates the energy of a molecule by acting on a qubit register that represents the molecule's quantum state by Jordan-Wigner encoding. If we prepare the molecule in the ground state, then we know that applying the Hamiltonian will calculate the ground state energy.

In [43]:
from qdk.chemistry.broombridge import load_and_encode

In [44]:
encoded_data_h2 = load_and_encode("../data/broombridge/hydrogen_0.2.yaml")

This contains a tuple of the number of qubits, the fermionic Hamiltonian term coefficients and the energy offset.

The index of each list in the second tuple is will be referred to as the "term type".

In [45]:
num_qubits, fermion_terms, _, energy_offset = encoded_data_h2

4 ([([0], [0.17120128499999998]), ([1], [0.17120128499999998]), ([2], [-0.222796536]), ([3], [-0.222796536])], [([0, 1], [0.1686232915]), ([0, 2], [0.12054614575]), ([0, 3], [0.16586802525]), ([1, 2], [0.16586802525]), ([1, 3], [0.12054614575]), ([2, 3], [0.1743495025])], [], [([0, 1, 2, 3], [0.0, -0.0453218795, 0.0, 0.0453218795])]) (2, [((1.0, 0.0), [0, 1])]) -0.09883444600000002


### 2. Resource estimation
Before running on hardware, let's first estimate the quantum resources needed for measuring two-electron terms ("term type" = 3)

In [32]:
GetHamiltonianTermH2.estimate_resources(
    nOp=12
)

AttributeError: 'GetHamiltonianTermH2' object has no attribute 'estimate_resources'

We have 11 qubits on the IonQ system so this is feasible to run!

### 3. Running on Azure Quantum

Connect to Azure Quantum

In [46]:
rid = "/subscriptions/9282e238-0ef1-4ebb-be4c-9399b8b2816f/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/rheaparekh-azure-quantum" # Enter your workspace's resource ID here
location = "West US 2" # Enter your workspace's location here, e.g. "West US"

In [47]:
qsharp.azure.connect(
   resourceId=rid,
   location=location,
   credential="devicecode"
)

Connecting to Azure Quantum...

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code SURHKFM2P to authenticate.


Connected to Azure Quantum workspace rheaparekh-azure-quantum in location westus2.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 5392},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 814384},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 60182},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current_availability': {}, 'average_queue_time': 906458},
 {'id': 'quantinuum.sim.h1-1sc', 'current_availability': {}, 'average_queue_time': 9106},
 {'id': 'quantinuum.qpu.h1-2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.sim.h1-2sc', 'current_availability': {}, 'average_queue_time': 382},
 {'id': 'quantinuum.sim.h1-1e', 'current_availability': {}, 'average_queue_time': 81379},
 {'id': 'quantinuum.sim.h1-2e', 'current_availability': {}, 'average_queue_time': 19621},
 {'id': 'rigetti.sim.qvm', 'current_availability': {}, 'average_queue_time': 5},
 {'id': 'rigetti.qpu.aspen-m-2', 'current_availabilit

In [48]:
qsharp.azure.target("ionq.simulator")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


{'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 60182}

Run a single term on the IonQ simulator:

In [47]:
n_op = 17
result = qsharp.azure.execute(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}")

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 17
   Job ID: cca2e176-5c2b-4157-9c4c-df8c35ed7294
Waiting up to 300 seconds for Azure Quantum job to complete...
[21:52:03] Current job status: Waiting
[21:52:08] Current job status: Succeeded


In [42]:
result

Result,Frequency,Histogram
0,0.5,
1,0.5,


### 4. Loop over all Hamiltonian terms

Each Hamiltonian term has a term type, measurement operator and coefficient. To simplify this sample, we will move the coefficients into a flat list. Each nonzero coefficient will render a separate job that we send to Azure Quantum. To calculate the energy for that term, we map result from the job onto a -1 to 1 axis and multiply the value by the corresponding coefficient.

In [49]:
coeffs = []
mesOps = []
for term_type, terms in enumerate(fermion_terms):
    print ("term_type", term_type)
    for (qubits, coeff) in terms:
        print("qubits", qubits, "coeff", coeff)
        coeffs += ExpandedCoefficients_(coeff=coeff, termType=term_type)
        mesOps += JordanWignerMeasurementOperators(nQubits=num_qubits, indices=qubits, termType=term_type)
# coeffs

# for n_op, coeff in enumerate(coeffs):
#  if coeff != 0.0:
#     print (n_op, coeff)

print(mesOps)

term_type 0
qubits [0] coeff [0.17120128499999998]
qubits [1] coeff [0.17120128499999998]
qubits [2] coeff [-0.222796536]
qubits [3] coeff [-0.222796536]
term_type 1
qubits [0, 1] coeff [0.1686232915]
qubits [0, 2] coeff [0.12054614575]
qubits [0, 3] coeff [0.16586802525]
qubits [1, 2] coeff [0.16586802525]
qubits [1, 3] coeff [0.12054614575]
qubits [2, 3] coeff [0.1743495025]
term_type 2
term_type 3
qubits [0, 1, 2, 3] coeff [0.0, -0.0453218795, 0.0, 0.0453218795]
[[2, 0, 0, 0], [0, 2, 0, 0], [0, 0, 2, 0], [0, 0, 0, 2], [2, 2, 0, 0], [2, 0, 2, 0], [2, 0, 0, 2], [0, 2, 2, 0], [0, 2, 0, 2], [0, 0, 2, 2], [1, 1, 1, 1], [3, 3, 3, 3], [1, 1, 3, 3], [3, 3, 1, 1], [1, 3, 1, 3], [3, 1, 3, 1], [3, 1, 1, 3], [1, 3, 3, 1]]


In [38]:
result = JordanWignerMeasurementOperators(nQubits=4, indices=[0, 1,2, 3], termType=3)
print(result)

[[1, 1, 1, 1], [3, 3, 3, 3], [1, 1, 3, 3], [3, 3, 1, 1], [1, 3, 1, 3], [3, 1, 3, 1], [3, 1, 1, 3], [1, 3, 3, 1]]


There are in total 18 coefficients corresponding to each combination of Hamiltonian term and Jordan-Wigner measurement operator. Start a job for each term that has a nonzero coefficient:

In [52]:
jobs = [
    qsharp.azure.submit(GetHamiltonianTerm, nOp=n_op, mesOps = mesOps[n_op], shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0.0
]

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 0
   Job ID: a7886f62-12a4-4759-a815-bfeee8abbe42
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 1
   Job ID: 222c4d60-e0fa-46ff-84da-b4d3de587b6a
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 2
   Job ID: efb6deb0-c854-431e-be47-0be32fd9c884


In [36]:
results = [qsharp.azure.output(j.id) for j in jobs]
results

NameError: name 'jobs' is not defined

### 5. Estimate energy

In [ ]:
def calc_energy(coeffs, results):
    return sum([(2. * res.get("0", 0.0) - 1.) * coeff for coeff, res in zip(coeffs, results)]) + energy_offset

In [ ]:
calc_energy(coeffs, results)

Now we are ready to run on hardware! Switch to the QPU.

In [ ]:
qsharp.azure.target("ionq.qpu")

In [ ]:
jobs_hw = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0
]

Check if the jobs succeeded.

In [ ]:
jobs_status = [qsharp.azure.status(j) for j in jobs_hw]
[
    j.status + " | " + j.name + " | " + j.target + " | " + j.creation_time + " | " + j.end_execution_time 
    for j in jobs_status
]

Get results and estimate energy

In [ ]:
results = [qsharp.azure.output(j.id) for j in jobs_hw]
results

In [ ]:
calc_energy(coeffs, results)